In [1]:
import os

os.chdir('/home1/awbai/analysis_data')

In [2]:
# 1. Import cptac
import cptac 

In [3]:
# 3. Download the ccrcc cancer data 
cptac.download(dataset="Ccrcc")

# 4. Load the breast cancer data set
ccrcc = cptac.Ccrcc()

cptac warning: Your version of cptac (1.1.2) is out-of-date. Latest is 1.5.7. Please run 'pip install --upgrade cptac' to update it. (/home1/awbai/.conda/envs/qbio_490_angela/lib/python3.9/threading.py, line 917)


In [4]:
protein_data = ccrcc.get_proteomics() # get the proteomics data
protein_data.columns = protein_data.columns.get_level_values(0) 
#protein_data 

In [5]:
rna_data = ccrcc.get_transcriptomics() # get the rna data
#rna_data

In [6]:
clinical_data = ccrcc.get_clinical() # get the rna data
#clinical_data

In [7]:
import numpy as np
import pandas as pd

#Preprocessing 
clinical_data = clinical_data[clinical_data["tumor_stage_pathological"].notna()]
ccrcc_index = clinical_data.index
protein_data = protein_data.loc[ccrcc_index]


In [8]:
#Segment tumor_stage_pathological into two cohorts, stage I and stage III

#Create a dataframe with just patient barcodes and gender
tumor_stage_df = clinical_data[["tumor_stage_pathological"]].copy()

#tumor_stage_df

#Create a mask selecting only stage 1
stage_1_mask = tumor_stage_df['tumor_stage_pathological'] == 'Stage I'

#stage_1_mask

#Create a mask selecting only stage 3
stage_3_mask = tumor_stage_df['tumor_stage_pathological'] == 'Stage III'

#stage_3_mask


In [9]:
#Identifying cohorts in protein_data
stage_1_protein_data = protein_data.loc[stage_1_mask]
#stage_1_protein_data 

stage_3_protein_data = protein_data.loc[stage_3_mask]
#stage_3_protein_data

In [10]:
#Finding difference between means of each protein
stage_1_protein_mean = np.mean(stage_1_protein_data, axis=0)
stage_1_protein_mean

stage_3_protein_mean = np.mean(stage_3_protein_data, axis=0)
stage_3_protein_mean

difference_protein_mean = (stage_1_protein_mean - stage_3_protein_mean)
difference_protein_mean

difference_protein_mean.sort_values()


Name
GDF6       -1.646855
PARP3      -1.201305
MXRA8      -1.103787
GPANK1     -1.098036
DCAF12L1   -1.097045
              ...   
ZNF195           NaN
ZNF275           NaN
ZNF507           NaN
ZNF71            NaN
ZNF823           NaN
Length: 11710, dtype: float64

In [17]:
#Top 5 Proteins: GDF6,PARP3,MXRA8,GPANK1,DCAF12L1

proteins = ['GDF6','PARP3','MXRA8','GPANK1','DCAF12L1']

top_proteins = protein_data[proteins]

print(top_proteins)


Name            GDF6     PARP3     MXRA8    GPANK1  DCAF12L1
Patient_ID                                                  
C3L-00004        NaN       NaN       NaN       NaN       NaN
C3L-00010        NaN       NaN       NaN       NaN       NaN
C3L-00011        NaN -0.199905       NaN       NaN       NaN
C3L-00026        NaN       NaN       NaN       NaN -0.526813
C3L-00079   1.831929  0.518124       NaN       NaN       NaN
...              ...       ...       ...       ...       ...
C3N-01646        NaN       NaN       NaN -0.268065       NaN
C3N-01648        NaN       NaN       NaN       NaN       NaN
C3N-01649        NaN       NaN       NaN       NaN       NaN
C3N-01651        NaN       NaN -0.506678       NaN       NaN
C3N-01808        NaN       NaN       NaN       NaN       NaN

[110 rows x 5 columns]


In [11]:
#Creating new dataframe of genes where rows are patients and columns are expression values

rna_data = rna_data.loc[ccrcc_index]
log_rna = np.log2(rna_data)

/home1/awbai/.conda/envs/qbio_490_angela/lib/python3.9/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [12]:
#Identifying stage 1 and stage 3 cohorts in rna_data
stage_1_rna_data = log_rna.loc[stage_1_mask]
#stage_1_rna_data 

stage_3_rna_data = log_rna.loc[stage_3_mask]
#stage_3_rna_data

In [13]:
#Finding difference between means of each gene
stage_1_rna_mean = np.mean(stage_1_rna_data, axis=0)
stage_1_rna_mean

stage_3_rna_mean = np.mean(stage_3_rna_data, axis=0)
stage_3_rna_mean

difference_rna_mean = (stage_1_rna_mean - stage_3_rna_mean)
difference_rna_mean

difference_rna_mean.sort_values()

Name
HIPK4     -inf
KCNA2     -inf
SFRP2     -inf
JSRP1     -inf
CLDN16    -inf
          ... 
ZSCAN10    NaN
ZSCAN4     NaN
ZSCAN5B    NaN
ZSCAN5C    NaN
ZSWIM2     NaN
Length: 19275, dtype: float64

In [14]:
#Top 5 Genes: HIPK4,KCNA2,SFRP2,JSRP1,CLDN16

top_genes = ['HIPK4','KCNA2','SFRP2','JSRP1','CLDN16']

rna_top_genes = log_rna[top_genes]
print(rna_top_genes)

Name           HIPK4     KCNA2     SFRP2     JSRP1    CLDN16
Patient_ID                                                  
C3L-00004  -5.271505 -4.704160  0.507348  0.420916 -3.880805
C3L-00010  -6.711398 -4.189856  1.821756 -4.603939 -3.806124
C3L-00011  -6.590781  0.148992  1.506826      -inf -4.007436
C3L-00026       -inf -4.674509 -3.292724 -2.871362 -1.308012
C3L-00079  -2.596586 -5.514668  1.406276 -2.126558 -2.013241
...              ...       ...       ...       ...       ...
C3N-01646  -4.463693 -3.404494 -0.983181 -2.548879 -2.420916
C3N-01648  -2.494321  1.769761  6.509188  5.104991 -5.232904
C3N-01649  -4.181524 -4.476675  3.075945 -2.811031 -2.832644
C3N-01651  -4.849088 -4.281742 -0.993067 -2.741629 -0.565303
C3N-01808  -4.179804 -6.612458 -4.908746 -4.072346 -6.596459

[110 rows x 5 columns]


In [49]:
#Create a list of the patients' cancer stages 
cancer_stage = clinical_data['tumor_stage_pathological']

#Making cancer_stage and rna_top_genes the same number of rows, NaN selected
cancer_mask = ['C3L-00097', 'C3L-00448', 'C3L-00796', 'C3L-01607', 'C3N-00313', 'C3N-01178']
stage_mask = ~cancer_stage.index.isin(cancer_mask)
cancer_stage = cancer_stage[stage_mask]

cancer_stage

Patient_ID
C3L-00004    Stage III
C3L-00010      Stage I
C3L-00011     Stage IV
C3L-00026      Stage I
C3L-00079    Stage III
               ...    
C3N-01646    Stage III
C3N-01648     Stage II
C3N-01649    Stage III
C3N-01651     Stage II
C3N-01808      Stage I
Name: tumor_stage_pathological, Length: 104, dtype: object

In [50]:
#Scaling and encoding

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

rna_top_genes.replace([np.inf, -np.inf], np.nan, inplace = True)
rna_top_genes = rna_top_genes.dropna()

cancer_stage = cancer_stage.dropna()

scaler = StandardScaler()
scaled_feature = scaler.fit_transform(rna_top_genes)

#Encoding
label = LabelEncoder()
label_encoder_features = label.fit_transform(cancer_stage)

#encoder = OrdinalEncoder()
#unencoded_columns = cancer_stages_list
#encoded_columns = encoder.fit_transform(unencoded_columns)
#cancer_stages_list = encoded_columns

#cancer_stages_list

print(rna_top_genes)
print(cancer_stage)

Name           HIPK4     KCNA2     SFRP2     JSRP1    CLDN16
Patient_ID                                                  
C3L-00004  -5.271505 -4.704160  0.507348  0.420916 -3.880805
C3L-00010  -6.711398 -4.189856  1.821756 -4.603939 -3.806124
C3L-00079  -2.596586 -5.514668  1.406276 -2.126558 -2.013241
C3L-00088  -4.140389 -4.194532 -1.103796 -3.354859 -4.294010
C3L-00096  -2.837184 -3.800353 -0.179102  0.739760 -0.824995
...              ...       ...       ...       ...       ...
C3N-01646  -4.463693 -3.404494 -0.983181 -2.548879 -2.420916
C3N-01648  -2.494321  1.769761  6.509188  5.104991 -5.232904
C3N-01649  -4.181524 -4.476675  3.075945 -2.811031 -2.832644
C3N-01651  -4.849088 -4.281742 -0.993067 -2.741629 -0.565303
C3N-01808  -4.179804 -6.612458 -4.908746 -4.072346 -6.596459

[104 rows x 5 columns]
Patient_ID
C3L-00004    Stage III
C3L-00010      Stage I
C3L-00011     Stage IV
C3L-00026      Stage I
C3L-00079    Stage III
               ...    
C3N-01646    Stage III
C3N-01648  

In [51]:
#Train Split Test (train_size = 0.7)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(rna_top_genes, cancer_stage, train_size=0.7, random_state=4)
print(X_train.shape) # dataset split by train size
print(X_test.shape) # dataset not included in above dataset
print(y_train.shape) # labels for corresponding X_train
print(y_test.shape) # labels for corresponding X_test



(72, 5)
(32, 5)
(72,)
(32,)


In [53]:
#Best model: 
#Testing accuracy of all four classifiers 

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

kneighbors_class = KNeighborsClassifier()
decisiontree_class = DecisionTreeClassifier()
mlp_class = MLPClassifier()
gaussianNB_class = GaussianNB()

runs = 10
accuracy = []
totalScore = 0

for _ in range(runs):
    X_train, X_test, y_train, y_test = train_test_split(rna_top_genes, cancer_stage, train_size=0.7, random_state=4)

#KNeighbors

kneighbors_class.fit(X_train, y_train)
knn_predictor = kneighbors_class.predict(X_test)
knn_accurate = accuracy_score(y_test, knn_predictor)
knNeighbors = kneighbors_class.score(np.array(X_test), y_test)

#DecisionTree

decisiontree_class.fit(X_train, y_train)
decisiontree_predictor = decisiontree_class.predict(X_test)
decisiontree_accurate = accuracy_score(y_test, decisiontree_predictor)
decisionTree = decisiontree_class.score(np.array(X_test), y_test)

#MLPClassifier

mlp_class.fit(X_train, y_train)
mlp_predictor = mlp_class.predict(X_test)
mlp_accurate= accuracy_score(y_test, mlp_predictor)
mlp = mlp_class.score(np.array(X_test), y_test)

#GaussianNB

gaussianNB_class.fit(X_train, y_train)
gaussianNB_predictor = gaussianNB_class.predict(X_test)
gaussianNB_accurate = accuracy_score(y_test, gaussianNB_predictor)
gaussianNB = gaussianNB_class.score(np.array(X_test), y_test)

print(knNeighbors)
print(decisionTree)
print(mlp)
print(gaussianNB)

#The best model is knNeighbors, followed by MLP Classifier, Gaussian NB, and Decision Tree!

0.34375
0.25
0.3125
0.28125
